In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
data = pd.read_csv('boston.csv')

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
pd.read_sql(
    """
    select
    sum(CRIM_N)
    ,sum(ZN_N)
    ,sum(INDUS_N)
    ,sum(CHAS_N)
    ,sum(NOX_N)
    from (
    SELECT
    case when CRIM is null then 1 else 0 end CRIM_N
    ,case when ZN is null then 1 else 0 end ZN_N
    ,case when INDUS is null then 1 else 0 end INDUS_N
    ,case when CHAS is null then 1 else 0 end CHAS_N
    ,case when NOX is null then 1 else 0 end NOX_N
    FROM boston
    )
    """,
    con,
)

,sum(CRIM_N),sum(ZN_N),sum(INDUS_N),sum(CHAS_N),sum(NOX_N)
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
pd.read_sql(
    """
    SELECT
    count(distinct CRIM)
    ,count(distinct ZN)
    ,count(distinct INDUS)
    ,count(distinct CHAS)
    ,count(distinct NOX)
    FROM boston
    """,
    con,
)

,count(distinct CRIM),count(distinct ZN),count(distinct INDUS),count(distinct CHAS),count(distinct NOX)
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
pd.read_sql(
    """
    SELECT
    CASE WHEN MIN(CRIM) OVER () = MEDIAN_CRIM THEN CRIM ELSE NULL END CRIM_
    ,CASE WHEN MIN(ZN) OVER () = MEDIAN_ZN THEN ZN ELSE NULL END ZN_
    ,CASE WHEN MIN(INDUS) OVER () = MEDIAN_INDUS THEN INDUS ELSE NULL END INDUS_
    ,CASE WHEN MIN(CHAS) OVER () = MEDIAN_CHAS THEN CHAS ELSE NULL END CHAS_
    ,CASE WHEN MIN(NOX) OVER () = MEDIAN_NOX THEN NOX ELSE NULL END NOX_
    FROM (
    SELECT *
    ,max(case when R_CRIM = ROUND(AVG_R_CRIM / 2) THEN CRIM ELSE 0 END) over () MEDIAN_CRIM
    ,max(case when R_ZN = ROUND(AVG_R_ZN / 2) THEN ZN ELSE 0 END) over () MEDIAN_ZN
    ,max(case when R_INDUS = ROUND(AVG_R_INDUS / 2) THEN INDUS ELSE 0 END) over () MEDIAN_INDUS
    ,max(case when R_CHAS = ROUND(AVG_R_CHAS / 2) THEN CHAS ELSE 0 END) over () MEDIAN_CHAS
    ,max(case when R_NOX = ROUND(AVG_R_NOX / 2) THEN NOX ELSE 0 END) over () MEDIAN_NOX
    FROM (
    SELECT *
    ,(max(R_CRIM) over()) as AVG_R_CRIM
    ,(max(R_ZN) over()) as AVG_R_ZN
    ,(max(R_INDUS) over()) as AVG_R_INDUS
    ,(max(R_CHAS) over()) as AVG_R_CHAS
    ,(max(R_NOX) over()) as AVG_R_NOX
    from (
    SELECT *
    ,RANK() over (order by CRIM) as R_CRIM
    ,RANK() over (order by ZN) as R_ZN
    ,RANK() over (order by INDUS) as R_INDUS
    ,RANK() over (order by CHAS) as R_CHAS
    ,RANK() over (order by NOX) as R_NOX
    FROM boston
    ) a
    ) b
    ) c
    --where cc <> 0
    """,
    con,
)

# Вывод: 50% всех домов находятся не возле реки и имеют минимальную долю земель под жилую застройку

,CRIM_,ZN_,INDUS_,CHAS_,NOX_
0,None,18.0,None,0.0,None
1,None,90.0,None,0.0,None
2,None,55.0,None,0.0,None
3,None,35.0,None,0.0,None
4,None,90.0,None,0.0,None
...,...,...,...,...,...
501,None,0.0,None,0.0,None
502,None,0.0,None,0.0,None
503,None,0.0,None,0.0,None
504,None,0.0,None,0.0,None


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
pd.read_sql(
    """
    with cte1 as (
    SELECT
     avg(RM) as room_expensive
    FROM boston
    where 1=1
    and MEDV = (select max(MEDV) from boston)
    ),
    cte25 as (
    SELECT avg(RM) as room_cheap_25 from (
    SELECT
    *
    FROM boston
    order by MEDV
    limit 25
    )a
    ),
    cte50 as (
    SELECT avg(RM) as room_cheap_50 from (
    SELECT
    *
    FROM boston
    order by MEDV
    limit 50
    )a
    ),
    cte100 as (
    SELECT avg(RM) as room_cheap_100 from (
    SELECT
    *
    FROM boston
    order by MEDV
    limit 100
    )a
    ),
    cte200 as (
    SELECT avg(RM) as room_cheap_200 from (
    SELECT
    *
    FROM boston
    order by MEDV
    limit 200
    )a
    ),
    cte300 as (
    SELECT avg(RM) as room_cheap_300 from (
    SELECT
    *
    FROM boston
    order by MEDV
    limit 300
    )a
    )
    
    
    select
    *
    -- На сколько в среденем
    --,diff_25/room_expensive
    --,diff_300/room_expensive
    --,((diff_100/room_expensive - diff_200/room_expensive)
    --+(diff_200/room_expensive - diff_300/room_expensive))*100/2 as mean_diff
    from (
    select
    room_expensive
    -- Разница между дорогими и дешевыми домами
    ,room_expensive - room_cheap_25 as diff_25
    ,room_expensive - room_cheap_50 as diff_50
    ,room_expensive - room_cheap_100 as diff_100
    ,room_expensive - room_cheap_200 as diff_200
    ,room_expensive - room_cheap_300 as diff_300
    from cte1
    cross join cte25
    cross join cte50
    cross join cte100
    cross join cte200
    cross join cte300
    ) a
    """,
    con,
)

#### Кол-во комнат влияет на стоимость. 
 ### Как сильно?????

,room_expensive,diff_25,diff_50,diff_100,diff_200,diff_300
0,7.484,1.73616,1.73076,1.59688,1.572295,1.511773


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех знаечений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [11]:
pd.read_sql(
    """
    SELECT LSTAT, MEDV, R_LSTAT FROM (
    SELECT
    *
    ,RANK() over (order by LSTAT) R_LSTAT
    FROM boston
    ) a
    WHERE 1=1
    and MEDV = (SELECT MAX(MEDV) FROM boston)
    """,
    con,
)

# Вывод: В соотношение признаков MEDV и LSTAT не прямопрослеживается взаимосвязь. Возможно на MEDV влияет третий
# признак, который подерживает высокую стоимость в районах с высокой процент населения с более низким статусом.

,LSTAT,MEDV,R_LSTAT
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
pd.read_sql(
    """
    select river, avg(MEDV) as mean_medv from (
    SELECT
    *
    ,case when chas = 1 then 'Граничит'
         when chas = 0 then 'Не граничит'
         else ''
         end as river
    FROM boston
    ) a
    group by river
    """,
    con,
)

,river,mean_medv
0,Граничит,28.440000
1,Не граничит,22.093843


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [13]:
pd.read_sql(
    """
    SELECT    
    CASE WHEN flag_crim <> 0 then CRIM else '' end CRIM
    ,CASE WHEN flag_zn <> 0 then CRIM else '' end ZN
    ,CASE WHEN flag_indus <> 0 then CRIM else '' end INDUS
    ,CASE WHEN flag_chas <> 0 then CRIM else '' end CHAS
    ,CASE WHEN flag_nox <> 0 then CRIM else '' end NOX
    FROM (
    SELECT
    *
    ,case when avg(CRIM) over() < avg(CRIM) over (partition by CHAS) then 1 else 0 end flag_crim
    ,case when avg(ZN) over() < avg(ZN) over (partition by CHAS) then 1 else 0 end flag_zn
    ,case when avg(INDUS) over() < avg(INDUS) over (partition by CHAS) then 1 else 0 end flag_indus
    ,case when avg(CHAS) over() < avg(CHAS) over (partition by CHAS) then 1 else 0 end flag_chas
    ,case when avg(NOX) over() < avg(NOX) over (partition by CHAS) then 1 else 0 end flag_nox
    FROM boston
    ) a
    where 1=1
    and CHAS = 1
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,,,3.32105,3.32105,3.32105
1,,,1.12658,1.12658,1.12658
2,,,1.41385,1.41385,1.41385
3,,,3.53501,3.53501,3.53501
4,,,1.27346,1.27346,1.27346
5,,,1.83377,1.83377,1.83377
6,,,1.51902,1.51902,1.51902
7,,,0.13587,0.13587,0.13587
8,,,0.43571,0.43571,0.43571
9,,,0.17446,0.17446,0.17446


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [26]:
pd.read_sql(
    """
    with cte1 as (
        SELECT
        *
        ,CAST(RANK() over (order by INDUS) AS float(10,5)) AS R_INDUS
        ,CAST(RANK() over (order by NOX) AS float(10,5)) AS R_NOX
        FROM boston
    ),
    cte2 as (
        select
        *
        ,(R_INDUS/(MAX(R_INDUS) over ()))*100 as PROC_R_INDUS
        ,(R_NOX/(MAX(R_NOX) over ()))*100 as PROC_R_NOX
        from cte1
    )
    
    
    Select * from (
    SELECT
    '10 proc' as type
    ,MAX(INDUS) as MAX_INDUS
    ,a.MAX_NOX
    FROM cte2
    cross join
    (
    SELECT MAX(NOX) as MAX_NOX FROM cte2 WHERE PROC_R_NOX <= 10
    ) a
    WHERE 1=1
    and PROC_R_INDUS <= 10
    UNION
    SELECT
    '20 proc' as type
    ,MAX(INDUS) as MAX_INDUS
    ,a.MAX_NOX
    FROM cte2
    cross join
    (
    SELECT MAX(NOX) as MAX_NOX FROM cte2 WHERE PROC_R_NOX <= 20
    ) a
    WHERE 1=1
    and PROC_R_INDUS <= 20
    
    UNION
    SELECT
    '30 proc' as type
    ,MAX(INDUS) as MAX_INDUS
    ,a.MAX_NOX
    FROM cte2
    cross join
    (
    SELECT MAX(NOX) as MAX_NOX FROM cte2 WHERE PROC_R_NOX <= 30
    ) a
    WHERE 1=1
    and PROC_R_INDUS <= 30
    
    UNION
    SELECT
    '40 proc' as type
    ,MAX(INDUS) as MAX_INDUS
    ,a.MAX_NOX
    FROM cte2
    cross join
    (
    SELECT MAX(NOX) as MAX_NOX FROM cte2 WHERE PROC_R_NOX <= 40
    ) a
    WHERE 1=1
    and PROC_R_INDUS <= 40
    
    UNION
    SELECT
    '50 proc' as type
    ,MAX(INDUS) as MAX_INDUS
    ,a.MAX_NOX
    FROM cte2
    cross join
    (
    SELECT MAX(NOX) as MAX_NOX FROM cte2 WHERE PROC_R_NOX <= 50
    ) a
    WHERE 1=1
    and PROC_R_INDUS <= 50
    
    UNION
    SELECT
    '60 proc' as type
    ,MAX(INDUS) as MAX_INDUS
    ,a.MAX_NOX
    FROM cte2
    cross join
    (
    SELECT MAX(NOX) as MAX_NOX FROM cte2 WHERE PROC_R_NOX <= 60
    ) a
    WHERE 1=1
    and PROC_R_INDUS <= 60
    
    UNION
    SELECT
    '70 proc' as type
    ,MAX(INDUS) as MAX_INDUS
    ,a.MAX_NOX
    FROM cte2
    cross join
    (
    SELECT MAX(NOX) as MAX_NOX FROM cte2 WHERE PROC_R_NOX <= 70
    ) a
    WHERE 1=1
    and PROC_R_INDUS <= 70
    
    UNION
    SELECT
    '80 proc' as type
    ,MAX(INDUS) as MAX_INDUS
    ,a.MAX_NOX
    FROM cte2
    cross join
    (
    SELECT MAX(NOX) as MAX_NOX FROM cte2 WHERE PROC_R_NOX <= 80
    ) a
    WHERE 1=1
    and PROC_R_INDUS <= 80
    
    UNION
    SELECT
    '90 proc' as type
    ,MAX(INDUS) as MAX_INDUS
    ,a.MAX_NOX
    FROM cte2
    cross join
    (
    SELECT MAX(NOX) as MAX_NOX FROM cte2 WHERE PROC_R_NOX <= 90
    ) a
    WHERE 1=1
    and PROC_R_INDUS <= 90
    
    UNION
    SELECT
    '100 proc' as type
    ,MAX(INDUS) as MAX_INDUS
    ,a.MAX_NOX
    FROM cte2
    cross join
    (
    SELECT MAX(NOX) as MAX_NOX FROM cte2 WHERE PROC_R_NOX <= 100
    ) a
    WHERE 1=1
    and PROC_R_INDUS <= 100
    ) c
    order by c.type
    """,
    con,
)

#  Вывод: наблюдается взаимосвязь между долей промышленной застройки(INDUS) и концентрации оксидов азота(NOX)

,type,MAX_INDUS,MAX_NOX
0,10 proc,2.89,0.426
1,100 proc,27.74,0.871
2,20 proc,4.05,0.439
3,30 proc,5.86,0.464
4,40 proc,6.96,0.504
5,50 proc,9.69,0.532
6,60 proc,12.83,0.550
7,70 proc,18.10,0.605
8,80 proc,18.10,0.647
9,90 proc,19.58,0.700
